In [1]:
import pickle
import os
import numpy as np
import pandas as pd

def ComputeRejectionRates(
    Model,
    Index,
    n,
    Lapscale,
    sig,
    Xdist="uniform",
    testtype="Wald",
    fixedU=False,
    randomBeta=False,
    alpha=0.05
):
    """
    Computes rejection rates for multiple time points (t=1 to t=5) based on p-value threshold.

    Parameters:
    - Model (str): Model type (e.g., "logistic").
    - Index (int): Setting index.
    - n (int): Sample size.
    - Lapscale (float): Laplace scale parameter.
    - sig (float): AR(1) parameter or independence.
    - Xdist (str): Distribution of X.
    - testtype (str): Test type (e.g., "Wald").
    - fixedU (bool): Whether U is fixed.
    - randomBeta (bool): Whether Beta is random.
    - alpha (float): Significance level for rejection.

    Returns:
    - List of rejection rates for t=1 to t=5.
    """
    # Construct file and directory paths
    fname = (
        f"{Model}_Setting_{Index}_n_{n}_Lapscale_{Lapscale}_sig_{sig}"
        f"_Xdist_{Xdist}_testtype_{testtype}_fixedU_{fixedU}_randomBeta_{randomBeta}"
    )
    results_dir = os.path.join(Model + "_results", "resultsH0", "data")
    input_file = os.path.join(results_dir, fname + "_dic.p")

    # Load p-value array
    with open(input_file, "rb") as infile:
        new_dict = pickle.load(infile)
    pV_array = new_dict["pV"]

    # Compute rejection rates for t=1 to t=5
    rejection_rates = [(pV_array[:, t - 1] < alpha).mean() for t in range(1, 6)]
    return rejection_rates

# Initialize parameters
ModelList = ["logistic"]
SettingsList = [1, 2, 3]
nList = [300, 2000]
sigList = [0, 0.1, 0.25]
LapscaleList = [0, 0.1, 0.5]
alpha = 0.05

# Prepare to store results
results = []

# Compute rejection rates for all scenarios
for Model in ModelList:
    for Index in SettingsList:
        for n in nList:
            for Lapscale in LapscaleList:
                for sig in sigList:
                    rejection_rates = ComputeRejectionRates(Model, Index, n, Lapscale, sig, alpha=alpha)
                    results.append({
                        "Setting": Index,
                        "n": n,
                        "Lapscale": Lapscale,
                        "sig": sig,
                        "t=1": rejection_rates[0],
                        "t=2": rejection_rates[1],
                        "t=3": rejection_rates[2],
                        "t=4": rejection_rates[3],
                        "t=5": rejection_rates[4],
                    })

# Convert results to DataFrame and save as CSV
results_df = pd.DataFrame(results)
output_file = "rejection_rates.csv"
results_df.to_csv(output_file, index=False)
print(f"Rejection rates saved to {output_file}")


Rejection rates saved to rejection_rates.csv
